# Import Libraries

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import datetime
import csv
import openpyxl
from openpyxl import Workbook

# Create Functions

## Scrape info

In [3]:
def MedtronicScraper(year,month,day):
    #prepare the input date
    year=str(year)
    month=str(month)
    day=str(day)
    input_date=year+'/'+month+'/'+day

    #open chrome in incognito mode
    options = webdriver.ChromeOptions()
    options.add_argument(' -- incognito')
    browser = webdriver.Chrome(chrome_options=options)

    # Go to the website
    browser.get('https://www.medtronic.com/jp-ja/about/news.html')

    # Get info
    # Go to production info section
    top_info = browser.find_element_by_xpath('/html/body/div[2]/div[5]/div[3]/section/div[2]/div/ul')
    # Go to the list
    news_list=top_info.find_elements_by_css_selector('li')
    # Go through the list
    result=[]
    for news in news_list:
        date = news.find_element_by_css_selector('p>b').text
        # Get URL and title if date == input date
        if date==input_date:
            # format without anything in between the date
            date = str(year)+str(month)+str(day)
            # Get link and title
            link=news.find_element_by_css_selector('p>a').get_attribute('href')
            title=news.find_element_by_tag_name('p').text.split('\n')[1]
            # If the title contains "販売"&"開始", them return 1 as new_product
            new_product_condition_1='販売'
            new_product_condition_2='開始'
            if (new_product_condition_1 in title)&(new_product_condition_2 in title):
                new_product=1
            else:
                new_product=0
            # Append the info to the list
            result.append([date,title,link,new_product])
    return result

## Store info to CSV

In [4]:
def Medtronic_to_csv(result):

    # check if the result is empty
    result_len = len(result)
    if result_len == 0:
        return
    
    # get date for checking their existence later
    date=result[0][0]

    # get row number
    # try to open the csv file
    try:
        with open('Medtronic.csv') as csvfile:
            reader = csv.reader(csvfile)
            # check if the title we are trying to add is already there    
            for row in reader:
                # the date is already there, dont add anything
                if row[0]==date:
                    return print('Already added to csv')
    # if there's no such file, create a new file 
    except FileNotFoundError:
        # set the header
        with open('Medtronic.csv','w') as file:
            header=['日付',
                    'カテゴリコード',
                    'メーカーコード',
                    'メーカー名称',
                    '新着記事カテゴリ',
                    '新着記事タイトル',
                    '新着記事URL',
                    '新製品記事'
                    ]
            writer = csv.writer(file)
            writer.writerow(header)
    
    # add new data
    with open('Medtronic.csv', 'a') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(result_len):
            writer.writerow([result[i][0], 
                            '', 
                            '', 
                            '日本メドトロニック', 
                            '',
                            result[i][1], 
                            result[i][2], 
                            result[i][3]])

## Store info to excel

In [5]:
def Medtronic_to_excel(result):
    # check if the result is empty
    result_len = len(result)
    if result_len == 0:
        return

    # get date for checking their existence later
    date=result[0][0]

    # try to open the workbook
    try:
        wb = openpyxl.load_workbook('Medtronic.xlsx')
        ws = wb['Sheet1']
        for row in ws.iter_rows(values_only=True):
            # the date is already there, dont add anything
            if row[0]==date:
                return print('Already added to excel')
    # if we cannot open it, we create a new one
    except FileNotFoundError:
        wb = Workbook()
        ws = wb.create_sheet('Sheet1')
        ws.append(['日付',
                'カテゴリコード',
                'メーカーコード',
                'メーカー名称',
                '新着記事カテゴリ',
                '新着記事タイトル',
                '新着記事URL',
                '新製品記事'])


    # check the last row in excel
    last_row = ws.max_row

    # update excel
    # can handle up to 3 news on the same day
    for i in range(result_len):
        ws.cell(row = last_row + i + 1, column = 1, value = result[i][0]) # 日付
        # add カテゴリコード、メーカーコード
        ws.cell(row = last_row + i + 1, column = 4, value = '日本メドトロニック') # メーカー名称
        # add 新着記事カテゴリ
        ws.cell(row = last_row + i + 1, column = 6, value = result[i][1]) # 新着記事タイトル
        ws.cell(row = last_row + i + 1, column = 7, value = result[i][2]) # 新着記事URL
        ws.cell(row = last_row + i + 1, column = 8, value = result[i][3]) # 新製品記事

    wb.save('Medtronic.xlsx')

In [7]:
if __name__=='__main__':
    year=2020
    month=6
    day=10
    result=MedtronicScraper(year,month,day)
    print(result)
    Medtronic_to_csv(result)
    Medtronic_to_excel(result)

[['2020610', '日本メドトロニック\u3000脊髄誘発電位測定等加算に関する診療報酬改定により、 術中神経モニタリング装置「NIM™システム3.0」の保険適用が拡大【詳細】', 'https://www.medtronic.com/jp-ja/about/news/pressrelease/2020-6-10.html', 0]]
Already added to csv
Already added to excel
